In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [19]:
df = pd.read_csv('data/stud.csv')

In [20]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [21]:
X =df.drop(columns = ['math_score'],axis=1)

In [22]:
y = df['math_score']

In [23]:
num_features = X.select_dtypes(exclude=object).columns
cat_features = X.select_dtypes(include=object).columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

preprocessing = ColumnTransformer(
    [
        ("OneHotEncoding",OneHotEncoder(),cat_features),
        ("StandardScaler",StandardScaler(),num_features),
    ]
)

In [24]:
X = preprocessing.fit_transform(X)

In [25]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [26]:
X.shape

(1000, 19)

In [27]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1)

In [28]:
X_train.shape,X_test.shape

((900, 19), (100, 19))

In [29]:
def evaluate_model(true,predicted):
    mse = mean_squared_error(true,predicted)
    r2_square = r2_score(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    mae = mean_absolute_error(true,predicted)
    return mae,rmse,r2_square

In [30]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 5.2955
- Mean Absolute Error: 4.2246
- R2 Score: 0.8773
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.6034
- Mean Absolute Error: 4.4408
- R2 Score: 0.8693


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.5253
- Mean Absolute Error: 5.1658
- R2 Score: 0.8137
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.8270
- Mean Absolute Error: 5.2649
- R2 Score: 0.8060


Ridge
Model performance for Training set
- Root Mean Squared Error: 5.2957
- Mean Absolute Error: 4.2239
- R2 Score: 0.8773
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.6010
- Mean Absolute Error: 4.4393
- R2 Score: 0.8694


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 5.7337
- Mean Absolute Error: 4.5436
- R2 Score: 0.8561
-----------------------

In [33]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
2,Ridge,0.869412
0,Linear Regression,0.869299
7,CatBoosting Regressor,0.842440
5,Random Forest Regressor,0.837367
6,XGBRegressor,0.815910
3,K-Neighbors Regressor,0.809538
8,AdaBoost Regressor,0.807634
1,Lasso,0.805985
4,Decision Tree,0.725555


In [35]:
lin_model = LinearRegression()
lin_model = lin_model.fit(X_train,y_train)
y_pred = lin_model.predict(X_test)
accuracy = r2_score(y_test,y_pred)*100
print( f"accuracy of Linear Regression is {accuracy} ")

accuracy of Linear Regression is 86.9298961927689 
